In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os,time,pickle
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from joblib import dump, load
from utils import data_process, visualize
from utils.utils import train, test, summary, setup_seed
from utils.models import Att_CNN, CNN, TFEQ, CRNN, M_CRNN

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [28]:
data = np.load("./data/data_train.npy", allow_pickle=True)
x_train = data.item().get("x_train")
y_train = data.item().get("y_train")
data = np.load("./data/data_test.npy", allow_pickle=True)
x_test = data.item().get("x_test")
y_test = data.item().get("y_test")
x_train=x_train.transpose((0,2,1))
x_test=x_test.transpose((0,2,1))


x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).long()

x_train_com, y_train_com, z_train_com = x_train[:, 0, :], x_train[:, 1, :], x_train[:, 2, :]
x_train_com = x_train_com.view(-1, 1, x_train_com.shape[-1])
y_train_com = y_train_com.view(-1, 1, y_train_com.shape[-1])
z_train_com = z_train_com.view(-1, 1, z_train_com.shape[-1])

x_test_com, y_test_com, z_test_com = x_test[:, 0, :], x_test[:, 1, :], x_test[:, 2, :]
x_test_com = x_test_com.view(-1, 1, x_test_com.shape[-1])
y_test_com = y_test_com.view(-1, 1, y_test_com.shape[-1])
z_test_com = z_test_com.view(-1, 1, z_test_com.shape[-1])

traindata_x = TensorDataset(x_train_com, y_train)
testdata_x = TensorDataset(x_test_com, y_test)
train_loader_x = DataLoader(traindata_x, batch_size=128, shuffle=True)
test_loader_x = DataLoader(testdata_x, batch_size=256, shuffle=True)

traindata_y = TensorDataset(y_train_com, y_train)
testdata_y = TensorDataset(y_test_com, y_test)
train_loader_y = DataLoader(traindata_y, batch_size=128, shuffle=True)
test_loader_y = DataLoader(testdata_y, batch_size=256, shuffle=True)

traindata_z = TensorDataset(z_train_com, y_train)
testdata_z = TensorDataset(z_test_com, y_test)
train_loader_z = DataLoader(traindata_z, batch_size=128, shuffle=True)
test_loader_z = DataLoader(testdata_z, batch_size=256, shuffle=True)

traindata = TensorDataset(x_train, y_train)
testdata = TensorDataset(x_test, y_test)
train_loader = DataLoader(traindata, batch_size=128, shuffle=True)
test_loader = DataLoader(testdata, batch_size=256, shuffle=True)

In [58]:
class Conv_com(nn.Module):
    def __init__(self, ):
        super(Conv_com, self).__init__()
        self.conv_1 = nn.Conv1d(
            in_channels=1,  out_channels=32, kernel_size=3, stride=2)
        self.conv_2 = nn.Conv1d(
            in_channels=32, out_channels=32, kernel_size=3, stride=2)
        self.conv_3 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=3)
        self.MaxPool = nn.MaxPool1d(2)
        self.rnn = nn.RNN(input_size=736, hidden_size=100)
        self.fla = torch.nn.Flatten()
        self.fc1 = nn.Linear(100, 2)

    def forward(self, x):
        out = F.relu(self.conv_1(x))
        out = F.relu(self.conv_2(out))
        out = F.relu(self.conv_3(out))
        out = self.MaxPool(out)
        out = self.fla(out)
        out = out.view(-1, 1, out.shape[-1])
        out, _ = self.rnn(out)
        out = self.fla(out)
        return out    

class M_CRNN(nn.Module):
    def __init__(self, ):
        super(M_CRNN, self).__init__()
        self.conv_x = Conv_com()
        self.conv_y = Conv_com()
        self.conv_z = Conv_com()
        self.rnn = nn.RNN(input_size=2208, hidden_size=100)
        self.fla = nn.Flatten()
        self.dp = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(300, 100)
        self.fc2 = nn.Linear(100, 2)

    def forward(self, data):
        x, y, z = data[:, 0, :], data[:, 1, :], data[:, 2, :]
        x = x.view(-1, 1, x.shape[-1])
        y = y.view(-1, 1, y.shape[-1])
        z = z.view(-1, 1, z.shape[-1])
        x_out = self.conv_x(x)
        y_out = self.conv_y(y)
        z_out = self.conv_z(z)
        new_feature = torch.cat([x_out, y_out, z_out], dim=1)
#         new_feature = new_feature.view(-1, 1, new_feature.shape[-1])
#         out, _ = self.rnn(new_feature)
        out = F.relu(self.fc1(new_feature))
        out = self.dp(out)
#         out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return F.log_softmax(out, dim=1)

In [66]:
%%time
setup_seed(42)
model = M_CRNN().cuda()
# model=TFEQ(channel=3, time_in=200).cuda()
L = []
test_hist = []
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 200 + 1):
    start = time.time()
    loss = train(model, train_loader, optimizer, epoch)
    L.append(loss)
    print("time {:.1f} sec:".format(time.time() - start))
    acc = test(model, test_loader)
    test_hist.append(acc)

Train Epoch: 1, Average Loss: 0.3860
time 2.1 sec:
Test set: Average loss: 0.3520, Accuracy: 1837/2070 (88.74%)

Train Epoch: 2, Average Loss: 0.3632
time 2.1 sec:
Test set: Average loss: 0.3531, Accuracy: 1837/2070 (88.74%)

Train Epoch: 3, Average Loss: 0.3612
time 2.1 sec:
Test set: Average loss: 0.3517, Accuracy: 1837/2070 (88.74%)

Train Epoch: 4, Average Loss: 0.3571
time 2.1 sec:
Test set: Average loss: 0.3518, Accuracy: 1837/2070 (88.74%)

Train Epoch: 5, Average Loss: 0.2851
time 2.1 sec:
Test set: Average loss: 0.1955, Accuracy: 1951/2070 (94.25%)

Train Epoch: 6, Average Loss: 0.1761
time 2.1 sec:
Test set: Average loss: 0.1404, Accuracy: 1989/2070 (96.09%)

Train Epoch: 7, Average Loss: 0.1644
time 2.1 sec:
Test set: Average loss: 0.1213, Accuracy: 1988/2070 (96.04%)

Train Epoch: 8, Average Loss: 0.1113
time 2.2 sec:
Test set: Average loss: 0.0915, Accuracy: 2009/2070 (97.05%)

Train Epoch: 9, Average Loss: 0.0865
time 2.4 sec:
Test set: Average loss: 0.0843, Accuracy: 201

Train Epoch: 73, Average Loss: 0.0127
time 2.3 sec:
Test set: Average loss: 0.0215, Accuracy: 2051/2070 (99.08%)

Train Epoch: 74, Average Loss: 0.0169
time 2.3 sec:
Test set: Average loss: 0.0182, Accuracy: 2056/2070 (99.32%)

Train Epoch: 75, Average Loss: 0.0100
time 2.3 sec:
Test set: Average loss: 0.0175, Accuracy: 2054/2070 (99.23%)

Train Epoch: 76, Average Loss: 0.0112
time 2.3 sec:
Test set: Average loss: 0.0183, Accuracy: 2052/2070 (99.13%)

Train Epoch: 77, Average Loss: 0.0130
time 2.2 sec:
Test set: Average loss: 0.0154, Accuracy: 2052/2070 (99.13%)

Train Epoch: 78, Average Loss: 0.0140
time 2.3 sec:
Test set: Average loss: 0.0154, Accuracy: 2056/2070 (99.32%)

Train Epoch: 79, Average Loss: 0.0167
time 2.3 sec:
Test set: Average loss: 0.0271, Accuracy: 2051/2070 (99.08%)

Train Epoch: 80, Average Loss: 0.0161
time 2.1 sec:
Test set: Average loss: 0.0190, Accuracy: 2057/2070 (99.37%)

Train Epoch: 81, Average Loss: 0.0109
time 2.1 sec:
Test set: Average loss: 0.0196, Accu

Train Epoch: 145, Average Loss: 0.0173
time 2.3 sec:
Test set: Average loss: 0.0133, Accuracy: 2057/2070 (99.37%)

Train Epoch: 146, Average Loss: 0.0096
time 2.3 sec:
Test set: Average loss: 0.0162, Accuracy: 2053/2070 (99.18%)

Train Epoch: 147, Average Loss: 0.0088
time 2.3 sec:
Test set: Average loss: 0.0116, Accuracy: 2060/2070 (99.52%)

Train Epoch: 148, Average Loss: 0.0095
time 2.3 sec:
Test set: Average loss: 0.0150, Accuracy: 2056/2070 (99.32%)

Train Epoch: 149, Average Loss: 0.0086
time 2.2 sec:
Test set: Average loss: 0.0148, Accuracy: 2058/2070 (99.42%)

Train Epoch: 150, Average Loss: 0.0105
time 2.1 sec:
Test set: Average loss: 0.0168, Accuracy: 2053/2070 (99.18%)

Train Epoch: 151, Average Loss: 0.0117
time 2.1 sec:
Test set: Average loss: 0.0201, Accuracy: 2056/2070 (99.32%)

Train Epoch: 152, Average Loss: 0.0069
time 2.1 sec:
Test set: Average loss: 0.0249, Accuracy: 2055/2070 (99.28%)

Train Epoch: 153, Average Loss: 0.0094
time 2.1 sec:
Test set: Average loss: 0.0

In [108]:
model.eval()
y_pred = []
out=[]
y_test = []
with torch.no_grad():
    for data, target in test_loader:
        data = data.cuda()
        output = model(data)
        output=F.softmax(output, dim=1)
        pred = output.max(1, keepdim=True)[1]  
        pred = pred.cpu().numpy().squeeze()
        y_pred.append(pred)
        out.append(output.cpu().numpy())
        y_test.append(target.numpy().squeeze())
y_pred=np.hstack(y_pred)
y_test=np.hstack(y_test)
out=np.vstack(out)

summary(y_pred,y_test)

confusion matrix:
 [[1829    4]
 [   8  229]]
Roc score: 0.9820
F1 score: 0.9745
Recall_score: 0.9662
Precision_score: 0.9828
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1833
           1       0.98      0.97      0.97       237

    accuracy                           0.99      2070
   macro avg       0.99      0.98      0.99      2070
weighted avg       0.99      0.99      0.99      2070



In [ ]:
torch.save(model, './models/M_CRNN_split.pth')

In [ ]:
earth=np.where(y_pred==1)[0]
non_earth=np.where(y_pred==0)[0]

p_earth=out[earth,1].squeeze()

p_non_earth=out[non_earth,0].squeeze()

In [ ]:
a = np.random.rand(100)
hist,bins = np.histogram(p_earth,bins=10,range=(0,1))
print(hist)
print(bins)


In [ ]:
a = np.random.rand(100)
hist,bins = np.histogram(p_non_earth,bins=10,range=(0,1))
print(hist)
print(bins)

In [34]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, classification_report, roc_auc_score,recall_score,precision_score
def summary(target, y_pred):
    print("confusion matrix:\n", confusion_matrix(target, y_pred))
    print('Roc score: %.4f' % roc_auc_score(target, y_pred))
    print("F1 score: %.4f" % f1_score(target, y_pred))
    print('Recall_score: %.4f' % recall_score(target, y_pred))
    print("Precision_score: %.4f" % precision_score(target, y_pred))
    print(classification_report(target, y_pred))